In [2]:
import igraph as ig

In [123]:
import csv
with open('page_names.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile)
    n = list(reader)
page_names = {}
for p in n:
    page_names[p[1]] = p[0]

In [46]:
with open('irish-likes.json') as data_file:    
  i_data = json.load(data_file)


In [125]:
nodes_users = []
for u in i_data.keys():
    if len(i_data[u]['likes']) > 5: nodes_users.append(u)
len(nodes_users)

252

In [126]:
nodes_posts = []
for u in nodes_users:
    for p in i_data[u]['likes']:
        post_id = p['post_id']
        if not post_id in nodes_posts: nodes_posts.append(post_id)
len(nodes_posts)

89

In [157]:
edges = []
for idx, val in enumerate(nodes_users):
    for p in i_data[val]['likes']:
        post_id = p['post_id']
        # add len(nodes_posts), assuming that id numbers of users begin after posts
        edges.append((len(nodes_posts)+idx,nodes_posts.index(post_id)))
len(edges)

1980

In [128]:
G=ig.Graph(edges, directed=False)

In [152]:
labels=[]
group=[]
for p in nodes_posts:
    labels.append(page_names[str(p)])
    group.append(1)
for u in nodes_users:
    labels.append(u)
    group.append(2)

In [153]:
layt=G.layout('kk', dim=3) 

In [160]:
N=len(nodes_users)+len(nodes_posts)
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]  
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]  


In [163]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             color=group,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Facebook users liking Irish referendum-related posts",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text="",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )


In [164]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

py.iplot(fig, filename='Irish referendum likes')
